In [65]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets/meirnizri/covid19-dataset/Covid Data.csv


In [66]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.tree import DecisionTreeClassifier

In [67]:
df = pd.read_csv('/kaggle/input/datasets/meirnizri/covid19-dataset/Covid Data.csv')

In [68]:
df.head()

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
0,2,1,1,1,03/05/2020,97,1,65,2,2,...,2,2,1,2,2,2,2,2,3,97
1,2,1,2,1,03/06/2020,97,1,72,97,2,...,2,2,1,2,2,1,1,2,5,97
2,2,1,2,2,09/06/2020,1,2,55,97,1,...,2,2,2,2,2,2,2,2,3,2
3,2,1,1,1,12/06/2020,97,2,53,2,2,...,2,2,2,2,2,2,2,2,7,97
4,2,1,2,1,21/06/2020,97,2,68,97,1,...,2,2,1,2,2,2,2,2,3,97


In [69]:
df.shape

(1048575, 21)

In [70]:


print(df.isnull().sum())



USMER                   0
MEDICAL_UNIT            0
SEX                     0
PATIENT_TYPE            0
DATE_DIED               0
INTUBED                 0
PNEUMONIA               0
AGE                     0
PREGNANT                0
DIABETES                0
COPD                    0
ASTHMA                  0
INMSUPR                 0
HIPERTENSION            0
OTHER_DISEASE           0
CARDIOVASCULAR          0
OBESITY                 0
RENAL_CHRONIC           0
TOBACCO                 0
CLASIFFICATION_FINAL    0
ICU                     0
dtype: int64


In [71]:
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

print("Numeric columns:", numeric_cols)
print("Categorical columns:", categorical_cols)

Numeric columns: Index(['USMER', 'MEDICAL_UNIT', 'SEX', 'PATIENT_TYPE', 'INTUBED', 'PNEUMONIA',
       'AGE', 'PREGNANT', 'DIABETES', 'COPD', 'ASTHMA', 'INMSUPR',
       'HIPERTENSION', 'OTHER_DISEASE', 'CARDIOVASCULAR', 'OBESITY',
       'RENAL_CHRONIC', 'TOBACCO', 'CLASIFFICATION_FINAL', 'ICU'],
      dtype='object')
Categorical columns: Index(['DATE_DIED'], dtype='object')


In [72]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder


df = pd.read_csv('/kaggle/input/datasets/meirnizri/covid19-dataset/Covid Data.csv')  


df.columns = df.columns.str.strip()


df_encoded = df.copy()
for col in df_encoded.select_dtypes(include=['object']).columns:
    df_encoded[col] = LabelEncoder().fit_transform(df_encoded[col])

correlation_matrix = df_encoded.corr()

loan_status_corr = correlation_matrix['ICU']
print(loan_status_corr)

USMER                   0.174901
MEDICAL_UNIT            0.205141
SEX                    -0.089844
PATIENT_TYPE           -0.975810
DATE_DIED               0.440904
INTUBED                 0.999434
PNEUMONIA               0.079691
AGE                    -0.317813
PREGNANT               -0.087991
DIABETES               -0.009223
COPD                   -0.020146
ASTHMA                 -0.023415
INMSUPR                -0.024077
HIPERTENSION           -0.007573
OTHER_DISEASE          -0.035330
CARDIOVASCULAR         -0.022413
OBESITY                -0.020254
RENAL_CHRONIC          -0.019285
TOBACCO                -0.022445
CLASIFFICATION_FINAL    0.193163
ICU                     1.000000
Name: ICU, dtype: float64


In [73]:
df.drop(columns=['INTUBED','CLASIFFICATION_FINAL','USMER','MEDICAL_UNIT'],inplace=True)

In [74]:
df.shape

(1048575, 17)

In [75]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['ICU']),
                                                 df['ICU'],
                                                 test_size=0.2,
                                                random_state=42)

In [76]:
X_train.head

<bound method NDFrame.head of         SEX  PATIENT_TYPE   DATE_DIED  PNEUMONIA  AGE  PREGNANT  DIABETES  \
408561    2             1  9999-99-99          2   62        97         1   
70143     2             1  9999-99-99          2   57        97         2   
708782    1             1  9999-99-99          2   34         2         2   
572694    2             1  9999-99-99          2   69        97         2   
774181    1             2  9999-99-99          2   59         2         2   
...     ...           ...         ...        ...  ...       ...       ...   
259178    1             1  9999-99-99          2   43         2         2   
365838    2             1  9999-99-99          2   29        97         2   
131932    2             1  9999-99-99          2   38        97         2   
671155    2             1  9999-99-99          2   46        97         2   
121958    1             1  9999-99-99          2   43         2         2   

        COPD  ASTHMA  INMSUPR  HIPERTENSION  

In [77]:
trf1 = ColumnTransformer([
    ('ohe_DATE_DIED', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), [4])
], remainder='passthrough')

In [78]:
trf2 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,15))
])

In [79]:
trf3 = SelectKBest(score_func=chi2,k=8)

In [80]:
trf4 = DecisionTreeClassifier(random_state=42)

In [81]:
pipe = make_pipeline(trf1,trf2,trf3,trf4)

In [82]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer-1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_DATE_DIED',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  [4])])),
                ('columntransformer-2',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 15, None))])),
                ('selectkbest',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x786ba086d620>)),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(random_state=42))])

In [83]:


pipe.named_steps



{'columntransformer-1': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_DATE_DIED',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse_output=False),
                                  [4])]),
 'columntransformer-2': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 15, None))]),
 'selectkbest': SelectKBest(k=8, score_func=<function chi2 at 0x786ba086d620>),
 'decisiontreeclassifier': DecisionTreeClassifier(random_state=42)}

In [84]:
from sklearn import set_config
set_config(display='diagram')

In [85]:
y_pred = pipe.predict(X_test)

In [86]:
y_pred

array([97, 97, 97, ..., 97, 97, 97])

In [87]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8098943804687314